# Calculate Accuracy for Mistral/textminr_ner
by Andreas Sünder & Benjamin Kissinger

## Load the Base Model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mistral-7B-Instruct-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,  # Mistral, same as before
    quantization_config=bnb_config,  # Same quantization config as before
    device_map="auto",
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_id, add_bos_token=True, torch_dtype=torch.float16, trust_remote_code=True)

## Load Finetuned Model

In [ ]:
from peft import PeftModel

ft_model = PeftModel.from_pretrained(base_model, "textminr/mistral-7b-4bit-ner")

## Load Dataset

In [5]:
from datasets import load_dataset
dataset = load_dataset('json', data_files="../augmentation/data/test.jsonl", split='train')
amount_examples = dataset.num_rows
amount_correct = 0

## Calculate Accuracy

In [ ]:
# model_input = tokenizer(eval_prompt3, return_tensors="pt").to("cuda")
answer = ""

for row in dataset:
    if len(row["prompt"]) > 150:
        model_input = tokenizer(row["prompt"], return_tensors="pt").to("cuda")
    else:
        model_input = tokenizer(row["prompt"] + row["prompt"], return_tensors="pt").to("cuda")
    ft_model.eval()
    with torch.no_grad():
        # TODO: Add noise in case of the input being too short
        x = tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=150, repetition_penalty=1.15)[0], skip_special_tokens=True)
        if dataset["response"] in x:
            amount_correct += 1

print("Accuracy: " + str(amount_correct / amount_examples))